### **1. Importing Liabraries**

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### **2. Loading Datasets**


In [3]:
data=pd.read_csv("../data/clean_data.csv",index_col=0)
data.head(10)
print("Dataset has " , data.shape[0] , " rows and ",data.shape[1] , " columns")
print('column names: \n') 
print('\n'.join(list(data.columns)))

Dataset has  1852394  rows and  29  columns
column names: 

trans_date_trans_time
cc_num
merchant
category
amt
first
last
gender
street
city
state
zip
lat
long
city_pop
job
dob
trans_num
unix_time
merch_lat
merch_long
is_fraud
label
age
trans_year
trans_month
trans_days
trans_week_days
trans_hour


In [4]:
data.dtypes

trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
label                     object
age                      float64
trans_year                 int64
trans_month                int64
trans_days                 int64
trans_week_days           object
trans_hour                 int64
dtype: object